In [2]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/836809378.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
pip install gurobipy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Create the optimization model
model = gb.Model("Can2Oil Transshipment Problem")

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
#Loading the data from CSV files
cost_production_to_refinement_df = pd.read_csv('/Users/pratiksha/Downloads/Cost_Production_to_Refinement.csv')
cost_production_to_transshipment_df = pd.read_csv('/Users/pratiksha/Downloads/Cost_Production_to_Transshipment.csv')
cost_transshipment_to_refinement_df = pd.read_csv('/Users/pratiksha/Downloads/Cost_Transshipment_to_Refinement.csv')
capacity_direct_production_df = pd.read_csv('/Users/pratiksha/Downloads/Capacity_for_Direct_Production_Facilities.csv')
capacity_transship_production_df = pd.read_csv('/Users/pratiksha/Downloads/Capacity_for_Transship_Production_Facilities.csv')
capacity_transship_distribution_df = pd.read_csv('//Users/pratiksha/Downloads/Capacity_for_Transship_Distribution_Centers.csv')
refinement_demand_df = pd.read_csv('/Users/pratiksha/Downloads/Refinement_Demand.csv')

In [6]:
#Print rows from CSV to check data is loaded correctly
print(cost_production_to_refinement_df.head())
print(cost_production_to_transshipment_df.head())
print(cost_transshipment_to_refinement_df.head())
print(capacity_direct_production_df.head())
print(capacity_transship_production_df.head())
print(capacity_transship_distribution_df.head())
print(refinement_demand_df.head())

   ProductionFacility  RefinementCenter      Cost
0                   1                 1  4.252733
1                   1                 2  4.567726
2                   1                 3  4.696484
3                   1                 4  2.678741
4                   1                 5  4.272451
   ProductionFacility  TransshipmentHub      Cost
0                   1                 1  2.378826
1                   1                 2  0.863842
2                   2                 1  1.666982
3                   2                 2  2.119488
4                   3                 1  2.174880
   TransshipmentHub  RefinementCenter      Cost
0                 1                 1  1.572329
1                 1                 2  3.465474
2                 1                 3  2.244062
3                 1                 4  3.773839
4                 1                 5  3.262652
   ProductionFacility  Capacity
0                   1       462
1                   2       103
2               

In [7]:
# A list of list of costs in matrix form to work on it
direct_shipping_costs_list = cost_production_to_refinement_df['Cost'].values.reshape(25, -1)
shipping_to_transsipment_costs_list = cost_production_to_transshipment_df['Cost'].values.reshape(15, -1)
transsipment_to_refinement_costs_list = cost_transshipment_to_refinement_df['Cost'].values.reshape(2, -1)

In [8]:
direct_shipping_costs_list

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [9]:
shipping_to_transsipment_costs_list

array([[2.37882586, 0.86384215],
       [1.66698156, 2.11948817],
       [2.17488041, 0.94818376],
       [2.67741553, 0.5924201 ],
       [2.82024783, 0.97232892],
       [2.85389005, 2.6943427 ],
       [1.40702817, 1.42832532],
       [2.48499907, 0.95347666],
       [1.78637201, 0.63669962],
       [0.68852771, 2.25970328],
       [2.40128579, 1.80010665],
       [0.56435019, 2.12484244],
       [1.98606958, 2.34717953],
       [1.63996858, 1.61928103],
       [0.64904663, 1.96881166]])

In [10]:
transsipment_to_refinement_costs_list

array([[1.57232851, 3.46547396, 2.24406225, 3.77383915, 3.2626518 ],
       [1.90042452, 1.6131232 , 1.70723241, 2.70539568, 2.25742842]])

In [11]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [12]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs_list[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs_list[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs_list[i][j]*z[i,j] for i in range(2) for j in range(5))
model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [13]:
capacity_direct_list = capacity_direct_production_df['Capacity'].values.reshape(25, -1)
capacity_transship_distribution_list = capacity_transship_distribution_df['Capacity'].values.reshape(2, -1)
capacity_transship_production_list = capacity_transship_production_df['Capacity'].values.reshape(15, -1)
refinement_demand_list = refinement_demand_df['Demand'].values.reshape(5, -1)

In [14]:
capacity_direct_list

array([[462],
       [103],
       [460],
       [325],
       [227],
       [217],
       [205],
       [521],
       [548],
       [191],
       [361],
       [411],
       [104],
       [155],
       [285],
       [109],
       [422],
       [438],
       [501],
       [139],
       [462],
       [504],
       [106],
       [132],
       [298]])

In [15]:
capacity_transship_distribution_list

array([[1317],
       [1453]])

In [16]:
capacity_transship_production_list

array([[374],
       [444],
       [395],
       [245],
       [378],
       [408],
       [435],
       [175],
       [415],
       [503],
       [184],
       [297],
       [450],
       [169],
       [365]])

In [17]:
refinement_demand_list

array([[1537],
       [1748],
       [1940],
       [1838],
       [1665]])

In [18]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(capacity_direct_list)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= capacity_direct_list[i], name=f"Direct Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/1095294607.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= capacity_direct_list[i], name=f"Direct Supply Constraint {i + 1}")


In [19]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(capacity_transship_production_list)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= capacity_transship_production_list[i], name=f"Transshipment Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/2848139284.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= capacity_transship_production_list[i], name=f"Transshipment Supply Constraint {i + 1}")


In [20]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= capacity_transship_distribution_list[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= capacity_transship_distribution_list[1], name="Transship Capacity 2")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/355622148.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= capacity_transship_distribution_list[0], name="Transship Capacity 1")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/355622148.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= capacity_transship_distribution_list[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [21]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [22]:
# Add the demand constraints
for k in range(len(refinement_demand_list)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand_list[k], name=f"Refinement Demand Constraint {k + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/4131438973.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand_list[k], name=f"Refinement Demand Constraint {k + 1}")


In [23]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x7f8fadb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [24]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

Number of Decision Variables:  165
Total Transportation cost:  24188.58516544768

    Variable            X 
-------------------------
Direct Shipping[0,3]          462 
Direct Shipping[1,1]          103 
Direct Shipping[2,2]          460 
Direct Shipping[4,3]           86 
Direct Shipping[5,1]          217 
Direct Shipping[7,4]          521 
Direct Shipping[8,4]          548 
Direct Shipping[10,4]          354 
Direct Shipping[11,0]            7 
Direct Shipping[11,2]          404 
Direct Shipping[12,0]          104 
Direct Shipping[13,4]          155 
Direct Shipping[14,3]          285 
Direct Shipping[15,0]          109 
Direct Shipping[17,1]          351 
Direct Shipping[17,4]           87 
Direct Shipping[18,3]          501 
Direct Shipping[20,2]          462 
Direct Shipping[21,3]          504 
Direct Shipping[22,1]          106 
Direct Shipping[23,2]          132 
Ship to Transshipment[0,1]          374 
Ship to Transshipment[2,1]          395 
Ship to Transshipment[3,1]        

In [25]:
# Extracting the decision variables for transshipment from India, Ukraine, and Germany to distribution centers
transship_to_italy = sum(z[0, j].x for j in range(5))  # Transshipment to Italy
transship_to_greece = sum(z[1, j].x for j in range(5))  # Transshipment to Greece

# Total amount of canola oil transshipped
total_transshipped = transship_to_italy + transship_to_greece

# Total demand for canola oil sourced from India, Ukraine, and Germany
total_demand_transshipped = sum(refinement_demand_list[j] for j in range(2, 5))

# Calculate proportion of canola oil transshipped
proportion_transshipped = total_transshipped / total_demand_transshipped

# Display the result
print("Proportion of canola oil transshipped:", proportion_transshipped)

Proportion of canola oil transshipped: [0.50891053]


In [26]:
# Create a new decision variable for transshipment flow
t = model.addVars(1, lb=0, vtype=GRB.CONTINUOUS, name="Total Transshipment")

# Modify the objective function to include the new variable
model.setObjective(
    direct_objective + trans_objective + trans_to_refinement_objective + t[0],
    GRB.MINIMIZE
)

# Add a constraint to limit the total transshipment
transshipment_limit = 100  # Adjust this value based on your specific constraint
model.addConstr(t[0] <= transshipment_limit, name="Transshipment Limit")

# Solve the modified model
model.optimize()

# Extract the optimal values of decision variables
optimal_values_x = model.getAttr('x', x)
optimal_values_y = model.getAttr('x', y)
optimal_values_z = model.getAttr('x', z)
optimal_value_t = model.getAttr('x', t)[0]

# Calculate the proportion of canola oil transshipped
total_transshipped = sum(optimal_values_y.values()) + sum(optimal_values_z.values()) + optimal_value_t
total_canola_oil = sum(optimal_values_x.values()) + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil

print(f"The proportion of canola oil transshipped is: {proportion_transshipped:.2%}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 166 columns and 361 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.418858517e+04
The proportion of canola oil transshipped is: 48.18%


In [27]:
# Create a new decision variable for transshipment flow
t = model.addVars(1, lb=0, vtype=GRB.CONTINUOUS, name="Total Transshipment")

# Add a constraint to limit the total transshipment
transshipment_limit = 100  # Adjust this value based on your specific constraint
model.addConstr(gb.quicksum(y[i, j] for i in range(15) for j in range(2)) + gb.quicksum(z[i, j] for i in range(2) for j in range(5)) <= transshipment_limit, name="Total Transshipment Limit")

# Solve the modified model
model.optimize()

# Extract the optimal values of decision variables
optimal_values_x = model.getAttr('x', x)
optimal_values_y = model.getAttr('x', y)
optimal_values_z = model.getAttr('x', z)
optimal_value_t = model.getAttr('x', t)[0]

# Calculate the proportion of canola oil transshipped
total_transshipped = sum(optimal_values_y.values()) + sum(optimal_values_z.values())
total_canola_oil = sum(optimal_values_x.values()) + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil

print(f"The proportion of canola oil transshipped is: {proportion_transshipped:.2%}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 51 rows, 167 columns and 401 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4188585e+04   5.440000e+03   0.000000e+00      0s

Solved in 19 iterations and 0.01 seconds (0.00 work units)
Infeasible model
The proportion of canola oil transshipped is: 1.14%


In [28]:
# Assuming you have loaded the data and created the initial model as described in the previous examples

# Create the Gurobi model
model = gb.Model("Canola_Oil_Optimization")

# Decision variables
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct_Shipping")
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship_to_Transshipment")
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment_to_Refinement")

# Objective function
direct_objective = gb.quicksum(direct_shipping_costs_list[i][j] * x[i, j] for i in range(25) for j in range(5))

# Introduce a discount for producers in North America
discount_factor = 0.8  # You can adjust this factor based on the desired preference
trans_objective = gb.quicksum(shipping_to_transsipment_costs_list[i][j] * y[i, j] * (1 if j < 3 else discount_factor) for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs_list[i][j] * z[i, j] for i in range(2) for j in range(5))

model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

# Capacity constraints
# Add the supply constraints from source nodes for direct shipping
for i in range(len(capacity_direct_list)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= capacity_direct_list[i], name=f"Direct Supply Constraint {i + 1}")

    
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(capacity_transship_production_list)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= capacity_transship_production_list[i], name=f"Transshipment Supply Constraint {i + 1}")

# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= capacity_transship_distribution_list[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= capacity_transship_distribution_list[1], name="Transship Capacity 2")


# Flow balance constraints
# Add the flow balance constraints for transshipment nodes
for j in range(5):
    model.addConstr(gb.quicksum(y[i, 0] for i in range(15)) - gb.quicksum(z[0, j] for i in range(2)) == 0, name=f"Flow Balance Italy {j + 1}")

for j in range(5):
    model.addConstr(gb.quicksum(y[i, 1] for i in range(15)) - gb.quicksum(z[1, j] for i in range(2)) == 0, name=f"Flow Balance Greece {j + 1}")


# Demand constraints
# Add the demand constraints
for k in range(len(refinement_demand_list)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand_list[k], name=f"Refinement Demand Constraint {k + 1}")


# Solve the modified model
model.optimize()

# Extract and analyze the optimal solution
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 57 rows, 165 columns and 480 nonzeros
Model fingerprint: 0xbaf8d896
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 47 rows, 155 columns, 460 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3375024e+03   1.255861e+03   0.000000e+00      0s
      32    2.3151783e+04   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.315178276e+04
Number of Decision Variables:  165
Total Transportation cost:  23151.78276419405

    Variable            X 
-------------------------
Direct_Shipping[0,3]          462 
Dir

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/3779299566.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= capacity_direct_list[i], name=f"Direct Supply Constraint {i + 1}")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/3779299566.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= capacity_transship_production_list[i], name=f"Transshipment Supply Constraint {i + 1}")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_1365/3779299566.py:32: DeprecationWarning: Conversion of an 